<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_4%EC%9D%BC%EC%B0%A8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2MB 51kB/s 
     |████████████████████████████████| 204kB 50.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=2207343436ac86afe7966d38753f9866d48be9dc0623f11fe53115dfb81ee6fa
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Titanic Binary Classification example") \
    .getOrCreate()

# 타이타닉 생존 예측 모델 만들기




In [3]:
spark

In [4]:
# 타이타닉 데이터 로컬로 다운로드
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv

--2021-02-04 18:23:49--  https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.235.81
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.235.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61197 (60K) [text/csv]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  59.76K  --.-KB/s    in 0.03s   

2021-02-04 18:23:50 (2.11 MB/s) - ‘titanic.csv’ saved [61197/61197]



In [5]:
# csv파일을 데이터프레임으로 만들기
# inferSchema=True하면 타입을 알아서 추정해준다.
data = spark.read.csv('./titanic.csv', header=True, inferSchema=True)
# 데이터프레임으로 만들었기 때문에 로컬 파이썬이 아닌 스파크 클러스터로 로딩된 것이다.

In [6]:
data.printSchema() # 타이타닉 데이터셋의 스키마(구조)출력
# 우리가 예측할 컬럼은 Survived이다.
# 1:생존, 0: 사망

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
data.show() # 타이타닉 데이터 20개 출력
# 숫자가 아닌 값, 카테고리컬한 값들이 많다.
# Age는 생존여부와 관련이 많아보이는데 비어있는게 많다. 중요한 컬럼은 무시하지 않고 결측치를 평균/중간값 등으로 채워야 한다.

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [8]:
data.select(['*']).describe().show()
# 판다스의 describe와 동일하게 spark에도 describe가 있다.
# 모든 데이터에 대해 describe
# 데이터셋의 여러 통계 정보를 볼 수 있다.
# Cabin은 쓰는게 별로 의미가 없음을 알 수 있다.

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|Gender|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

**데이터 클린업**: 

*   PassengerID, Name, Ticket, Embarked는 사용하지 않을 예정 (아무 의미가 없음).
*   Cabin도 비어있는 값이 너무 많아서 사용하지 않을 예정
*   Age는 중요한 정보인데 비어있는 레코드들이 많아서 디폴트값을 채워줄 예정
*   Gender의 경우 카테고리 정보이기에 숫자로 인코딩 필요



In [9]:
# 데이터프레임의 select 함수를 사용해서 내가 필요한 컬럼들만 들어간 새로운 데이터프레임 생성
final_data = data.select(['Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare'])

In [10]:
final_data.show() # 앞의 20개만 출력

+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
|       1|     1|female|35.0|    1|    0|   53.1|
|       0|     3|  male|35.0|    0|    0|   8.05|
|       0|     3|  male|null|    0|    0| 8.4583|
|       0|     1|  male|54.0|    0|    0|51.8625|
|       0|     3|  male| 2.0|    3|    1| 21.075|
|       1|     3|female|27.0|    0|    2|11.1333|
|       1|     2|female|14.0|    1|    0|30.0708|
|       1|     3|female| 4.0|    1|    1|   16.7|
|       1|     1|female|58.0|    0|    0|  26.55|
|       0|     3|  male|20.0|    0|    0|   8.05|
|       0|     3|  male|39.0|    1|    5| 31.275|
|       0|     3|female|14.0|    0|    0| 7.8542|
|       1|     2|female|55.0|    0|    0|   16.0|
|       0|     3|  male| 2.0|    4|    1| 29.125|


Age는 평균값으로 채운다

In [11]:
from pyspark.ml.feature import Imputer

# missing value가 있을 경우 평균 값으로 채운다. 대상은 Age 컬럼
# 평균값으로 채워진 새로운 컬럼의 이름은 AgeImputed이다.
imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])

# 위에서 만든 임퓨터로 실제 데이터프레임을 넣어 변환을 해주는 객체 imputer_model를 만든다.
# fit을 함으로써 인자로 들어가는 데이터프레임을 imputer로 만든 행위를 적용한다.
imputer_model = imputer.fit(final_data)

# transform함수의 인자로 들어온 데이터프레임에 새로운 칼럼을 추가한 것을 리턴한다.
# AgeImputed라는 칼럼이 새로 추가되게 된다.
final_data = imputer_model.transform(final_data)

In [12]:
final_data.select("Age", "AgeImputed").show()
# Age와 AgeImputed만 출력

+----+-----------------+
| Age|       AgeImputed|
+----+-----------------+
|22.0|             22.0|
|38.0|             38.0|
|26.0|             26.0|
|35.0|             35.0|
|35.0|             35.0|
|null|29.69911764705882|
|54.0|             54.0|
| 2.0|              2.0|
|27.0|             27.0|
|14.0|             14.0|
| 4.0|              4.0|
|58.0|             58.0|
|20.0|             20.0|
|39.0|             39.0|
|14.0|             14.0|
|55.0|             55.0|
| 2.0|              2.0|
|null|29.69911764705882|
|31.0|             31.0|
|null|29.69911764705882|
+----+-----------------+
only showing top 20 rows



성별 정보 인코딩: male -> 0, female -> 1

In [13]:
from pyspark.ml.feature import StringIndexer
# StringIndexer라는 트랜스포머로 문자열 값을 숫자로 바꿔줄 수 있다.

# Gender컬럼을 넣어 숫자로 변화한 뒤 그 결과를 GenderIndexed라는 데이터프레임으로 저장
gender_indexer = StringIndexer(inputCol='Gender', outputCol='GenderIndexed')

# fit으로 데이터프레임을 입력으로 해 위의 연산을 수행
gender_indexer_model = gender_indexer.fit(final_data)

# 최종적으로 transform을 통해 입력 데이터프레임에 위의 fit을 적용한 새로운 데이터프레임 컬럼을 추가
final_data = gender_indexer_model.transform(final_data)

In [15]:
final_data.select("Gender", "GenderIndexed").show()
# male은 0, female은 1로 바뀜

+------+-------------+
|Gender|GenderIndexed|
+------+-------------+
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
|  male|          0.0|
|  male|          0.0|
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
|  male|          0.0|
|female|          1.0|
|female|          1.0|
+------+-------------+
only showing top 20 rows



## 피쳐 벡터를 만들기

In [16]:
from pyspark.ml.feature import VectorAssembler

# 벡터어셈블러를 이용해 컬럼들을 하나의 벡터로 만들어 저장
# 예측해야하는 Survived를 빼고 벡터어셈블러 적용한 결과를 features라는 컬럼으로 저장함.
assembler = VectorAssembler(inputCols=['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeImputed', 'GenderIndexed'], outputCol='features')
data_vec = assembler.transform(final_data)

In [19]:
data_vec.show()

+--------+------+------+----+-----+-----+-------+-----------------+-------------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|GenderIndexed|            features|
+--------+------+------+----+-----+-----+-------+-----------------+-------------+--------------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|          0.0|[3.0,1.0,0.0,7.25...|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|          1.0|[1.0,1.0,0.0,71.2...|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|          1.0|[3.0,0.0,0.0,7.92...|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|          1.0|[1.0,1.0,0.0,53.1...|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|          0.0|[3.0,0.0,0.0,8.05...|
|       0|     3|  male|null|    0|    0| 8.4583|29.69911764705882|          0.0|[3.0,0.0,0.0,8.45...|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|      

## 훈련용과 테스트용 데이터를 나누고 binary classification 모델을 하나 만든다

In [20]:
# 훈련셋 70%, 테스트셋 30%
train, test = data_vec.randomSplit([0.7, 0.3])

In [21]:
from pyspark.ml.classification import LogisticRegression

# 인풋 피쳐는 features칼럼, 예측해야하는 칼럼은 Survived
algo = LogisticRegression(featuresCol="features", labelCol="Survived")

# 훈련데이터프레임을 주고 fit으로 훈련
model = algo.fit(train)

## 모델 성능 측정

In [23]:
# model의 transform은 prediction이라는 새로운 칼럼을 만든다.
predictions = model.transform(test)

In [24]:
predictions.select(['Survived','prediction', 'probability']).show()
# prediction이라는 새로운 칼럼이 추가된 것을 볼 수 있다. 
# logistic regression의 경우 확률값이 들어온다. 0~1사이의 값.
# 0.5가 threshold임을 알 수 있다. 
# 다른 threshold값을 쓰고싶으면 prediction칼럼값을 무시하고 probability 칼럼만 가지고 새로운 threshold를 적용해서 판단하면 된다.

+--------+----------+--------------------+
|Survived|prediction|         probability|
+--------+----------+--------------------+
|       0|       1.0|[0.01837440750260...|
|       0|       1.0|[0.04731535958683...|
|       0|       0.0|[0.52897256180839...|
|       0|       1.0|[0.49206966168615...|
|       0|       1.0|[0.48730578224404...|
|       0|       1.0|[0.47849932658827...|
|       0|       1.0|[0.46946879952632...|
|       0|       1.0|[0.46709542195481...|
|       0|       0.0|[0.50049776488273...|
|       0|       1.0|[0.23615259155964...|
|       0|       1.0|[0.32226404472933...|
|       0|       0.0|[0.55289102591406...|
|       0|       1.0|[0.49913264844792...|
|       0|       0.0|[0.59068399630842...|
|       0|       0.0|[0.54788012119119...|
|       0|       0.0|[0.63484230548410...|
|       0|       0.0|[0.65444652242197...|
|       0|       0.0|[0.60590751680479...|
|       0|       0.0|[0.66072838022720...|
|       0|       0.0|[0.64615983124768...|
+--------+-

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# BinaryClassificationEvaluator를 통해 Survived를 바탕으로 areaUnderROC를 계산한다.
# 1에 가까울수록 퍼포먼스가 좋은 것이다.
evaluator = BinaryClassificationEvaluator(labelCol='Survived', metricName='areaUnderROC')
evaluator.evaluate(predictions)

0.8256054421768714